In [ ]:
import pandas as pd
import os

from skimage import io
import numpy as np
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style = "dark", 
        color_codes = True,
        font_scale = 1.5)

# Data Augmentation

- increase the sample of non-tumor instances
- flip/rotate/cropping/alter color/image overlay

# Image Preprocessing

- Convert BGR TO GRAY
- GaussianBlur
- Threshold
- Erode
- Dilate
- Find Contours

Filtering: remove gaussian noise, salt-and-pepper noise, speckle noise; by median/mean/gaussian filter
- Gaussian smoothing, median filtering, and wavelet denoising

Enhancement: adjust brightness/contrast
- histogram equalization, adaptive histogram equalization, and contrast stretching

Restoration: deconvolution/denoising

Image resizing
- smaller/bigger/alter ratio
- nearest neighbor interpolation, bilinear interpolation, and bicubic interpolation

Colour correction
- gray world assumption, white balance, and color transfer

Segmentation
- thresholding, edge detection, and region growing


In [ ]:
# Convert BGR TO GRAY: reduce dimension
# GaussianBlur: reduce noise
# Threshold: only become white if the color intensity exceed the threshold, otherwise black

# Erode: remove border of foreground object, reduce image features (can be iteratively)
# Dilate: increase object area, emphasise features

# Uses of Erosion and Dilation: 

# Erosion: 
# It is useful for removing small white noises.
# Used to detach two connected objects etc.
# Dilation:
# In cases like noise removal, erosion is followed by dilation. Because, erosion removes white noises, but it also shrinks our object. So we dilate it. Since noise is gone, they won’t come back, but our object area increases.
# It is also useful in joining broken parts of an object.

# Find Contours, to perform cropping

# image resize for to 256/512?

# normalise to 0-1

# contrast: equalizeHist () method enhances the contrast of images.

In [6]:
import numpy as np
import cv2

# im = cv2.imread(r'D:\Users\dxlee\DX_degree\Y3S2\ODL\working-dir\brain-tumor\Testing\glioma_tumor\image.jpg')
im = cv2.imread(r'Testing\glioma_tumor\image.jpg')

# To Grayscale
im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)

print(np.shape(im))
data = np.asarray(im)
# for row in data:
#     for cell in row:
#         if cell !=0:
#             print(cell)
# print(data[np.any(data!=0, axis=1)])

cv2.imwrite("grayscale.jpg", im)

(619, 495)


True